# Cloud Metrics

In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import pdal

from forest_structure_tools.cloud_metrics import height_metrics, quantile_metrics, complexity_metrics
from forest_structure_tools.utils import add_suffix

In [2]:
data_dir = Path("../data")
plots_lidar_dir = data_dir / "outputs" / "plots" / "lidar"

In [3]:
def read_plot(plot_id: str):
    file_path = plots_lidar_dir / f"{plot_id}.copc.laz"
    pl = pdal.Reader(str(file_path)).pipeline()
    pl.execute()
    return pl.arrays[0]

In [4]:
points = read_plot("AGG_O_01_P1")

In [5]:
z = points['Z']
rn = points['ReturnNumber']
nor = points['NumberOfReturns']
weights = points['Weight']

In [6]:
pd.Series({ 
    **height_metrics(z),
    **height_metrics(z[z > 0], suffix='veg')
})

max           16.275999
min            0.000000
range         16.275999
mean           4.428016
median         4.555000
max_veg       16.275999
min_veg        0.001000
range_veg     16.275000
mean_veg       5.646877
median_veg     6.400000
dtype: float32